# Comparações de Algoritmos

Frequentemente precisamos comparar duas estratégias de resolução de um problema.
Essas comparações podem ser teóricas e computacionais.
Os dois aspectos mais importantes de comparação para métodos de otimização não-linear
são

  - O método converge?
  - Se sim, quão rápido?

Essas perguntas podem ser exploradas sob condições específicas (e normalmente são),
mas a ideia básica é essa.

## Método, Algoritmo, Implementação

Um aspecto importante da comparação é saber diferenciar o que é que você está
comparando. Esse detalhe muitas vezes é ignorado, supõe-se que despropositalmente,
o que acaba gerando conclusões parciais e subjetivas.

Um aspecto grandemente deixado de lado é qual a diferença entre método, algoritmo
e implementação. Vou definir os três e indicar de onde tirei cada definição.

  - **Método:** Um procedimento ou processo para obter um objetivo (Tradução livre do Merriam-Webster Online)
  - **Algoritmo:** Como um método mas com 5 características importantes:
    - Finitude (O método para de rodar)
    - Definição (Cada passo está bem definido)
    - Entrada
    - Saída
    - Eficiência (Cada passo deveria poder ser feito por um humano com lápis e papel em tempo finito)
    (Paráfrase de "The Art of Computer Programming, v.1" de Donald Knuth.)
  - **Implementação:** A realização física da tradução de um método ou algoritmo
    para uma linguagem de programação de maneira que o mesmo possa ser executado. (Eu)

É importante diferenciar aqui o método e o algoritmo, principalmente no âmbito da otimização.
A iteração de BFGS é um método. A atualização da matriz de BFGS é um algoritmo. O código que escrevemos é uma
implementação do método e do algoritmo.

Também é muito importante notar nós sempre queremos que o método seja um algoritmo, i.e.,
essencialmente queremos que o algoritmo pare.
Por exemplo,

  1. Dados $x_0 \in \mathbb{R}^n$ e $f:\mathbb{R}^n\rightarrow\mathbb{R}$
  2. Faça $k = 0$.
  3. Calcule $d^k = -\nabla f(x^k)$
  4. Calcule $\alpha_k = \arg\min_{\alpha > 0} f(x^k + \alpha d^k)$
  5. Faça $x^{k+1} = x^k + \alpha_k d^k$
  6. Faça $k = k + 1$.
  7. Se $\nabla f(x^k) = 0$ PARE com $x^* = x^k$.
       Senão, vá para o passo 3.

É um método, mas não um algoritmo, pois o algoritmo não acaba num número finito de iterações,
mesmo que a sequência seja convergente (por exemplo $f(x) = x_1^2 + 4x_2^2$ e $x^0 = (1,1)$).

Se mudamos a condição do passo 7 para $\Vert \nabla f(x^k)\Vert < 10^{-5}$,
ainda não temos um algoritmo, pois a sequência pode divergir (por exemplo $f(x) = -x^2$ e $x^0 = 1$).

Se mudamos a condição do passo 7 para $k > 1000$,
já chegamos bem perto de um algoritmo, se tudo é calculado corretamente, então $k$ chega a 1001.
Mas como o texto já diz, tudo precisa ser calculado corretamente. Isso quer dizer que $f$ é
diferenciável, e que a obtenção de $\alpha_k$ é possível. A primeira hipótese é trivial de
acrescentar, mas a segunda não, e também precisaríamos de alguma condição para ela.

Essas pequenas mudanças e variações são aquelas que fizemos no começo, para que o algoritmo parasse.
Algumas delas ignoramos, pois supomos que o método estaria funcionando. Mas se formos ser
estritos, precisamos nos preocupar com algumas coisas, como se o parâmetro da busca linear de Armijo
será suficientemente positivo, ou o raio da região de confiança, ou se Pred > 0, etc.
Nesses pontos é que é importante a teoria. Pois se provamos matematicamente que algo é possível sob
algumas hipóteses, podemos restringir nossas comparações à essas hipóteses, e colocar salva-guardas
caso elas falhem.

O algoritmo é a implementação de alguma ideia. O problema, como você pode notar, é que a ideia
nunca está sozinha. Dessa maneira, ao implementar o método de BFGS para as direções, também
implementamos a atualização de BFGS e a solução do sistema linear.
Ao comparar dois *métodos*, é preciso estar atento ao algoritmo em si, isto é, quais as condições
de parada embutidas, e também às implementações, e.g., como são resolvidos os sistemas lineares.
A atenção tem que ser redobrada ainda, pois não necessariamente o autor do livro ou artigo que você
está lendo está fazendo essa distinção.

## Comparando

Alguns problemas são facilmente resolvidos. Por exemplo, dado um vetor, como podemos ordená-lo?
Existem vários algoritmos que fazer essa tarefa, e para compará-los basta ver qual leva mais tempo,
ou mais iterações, pois são métricas mais ou menos equivalentes.
No entanto, na comparação de métodos de otimização, já começamos com o problema de se **o método
funciona** para o problema.

Como vimos, por exemplo, o método de Newton falha se encontrar uma hessiana indefinida ou singular.
Os outros métodos também podem falhar, em condições parecidas. Então, já não temos a garantia de que
o método funciona. Então, um parâmetro de comparação entre os métodos seria se eles convergem para
uma determinada seleção de problemas.

Isso não é suficiente, no entanto. Pois, também como vimos, o método de Newton é muito mais rápido
que o método do gradiente, em vários casos. Então, para os casos onde ele funciona, ele é mais
eficiente. Esse é outro parâmetro de comparação.

A questão agora é, qual é mais importante.

 - O método A resolve os problemas 80% mais rápido que o método B
 - O método B resolve 80% problemas a mais que o método A

Qual é melhor? E se no lugar de 80% tivermos 10%?

Essa pergunta depende de vários fatores e é subjetiva, então vamos fazer uma análise olhando para
várias possibilidades de **robustez** e **eficiência**. Essa análise será feita com o perfil de
desempenho (performance profile).

## Perfil de desempenho

Vamos considerar um conjunto $\mathcal{S}$ de algoritmos (ou métodos, ou implementações), e um
conjunto $\mathcal{P}$ de problemas que serão usados na comparação desses algoritmos.
Definimos como $t_{s,p}$ o tempo (ou número de avaliações de função, ou iterações, ou custo, etc.)
que o algoritmo $s$ leva para resolver o problema $p$.
Caso o algoritmo $s$ não consiga resolver o problema $p$, definimos $t_{s,p} = t_{\max}$,
de maneira que $t_{s,p} < t_{\max}$ para todo $s$ e $p$.

Definimos a razão de desempenho
$$ r_{s,p} = \frac{t_{s,p}}{\min{t_{s,p}}}, $$
que servirá para comparar os custos sem depender da escala de tempo.
Note que é necessário que $t_{s,p} > 0$, o que é natural, e também que
$r_{s,p} \geq 1$. Definimos $r_M > 0$ como o valor tal que
$r_{s,p} < r_M$.

Definimos
$$ \rho_s(\tau) = \frac{1}{N_P}\#\{p \in \mathcal{P} \mid r_{s,p} \leq \tau\}, $$
um tipo de distribuição acumulada do algoritmo $s$, que indica quando problemas o algoritmo $s$
consegue resolver com razão de desempenho até $\tau$.

O gráfico de perfil de desempenho consiste dos gráficos das funções $\rho_s$ para cada $s$. O valor
$\rho_s(1)$ indica quantos problemas o algoritmo $s$ consegue resolver tão rápido quanto o melhor
algoritmo, e o valor $$\lim_{\tau\rightarrow r_M^-} \rho_{s}$$
é a quantidade total de problemas que $s$ resolve. Esses valores são ditos eficiência e robustez,
respectivamente.

### Exemplo

Vamos simular três algoritmos.

In [63]:
# 10 problemas
NP = 10
# Função pra criar aleatório
rnd() = round(rand(NP)+0.01,2)
# Tempos que os algoritmos levaram
A = rnd(); B = rnd(); C = rnd();

In [64]:
@printf("%4s %6s %6s %6s\n", "Prob", "Alg A", "Alg B", "Alg C")
for i = 1:NP
    @printf("%4d %6.3f %6.3f %6.3f\n", i, A[i], B[i], C[i])
end

Prob  Alg A  Alg B  Alg C
   1  0.760  0.570  0.060
   2  0.790  0.240  0.760
   3  0.590  0.350  0.630
   4  0.710  0.740  0.950
   5  0.600  0.870  0.760
   6  0.410  0.290  0.390
   7  0.920  0.320  0.790
   8  0.880  0.100  0.560
   9  0.950  0.400  0.630
  10  0.900  0.330  0.690


In [65]:
@printf("%4s %9s %9s %9s\n", "Prob", "Alg A", "Alg B", "Alg C")
for i = 1:NP
    @printf("%4d %9.2e %9.2e %9.2e\n", i, A[i], B[i], C[i])
end

Prob     Alg A     Alg B     Alg C
   1  7.60e-01  5.70e-01  6.00e-02
   2  7.90e-01  2.40e-01  7.60e-01
   3  5.90e-01  3.50e-01  6.30e-01
   4  7.10e-01  7.40e-01  9.50e-01
   5  6.00e-01  8.70e-01  7.60e-01
   6  4.10e-01  2.90e-01  3.90e-01
   7  9.20e-01  3.20e-01  7.90e-01
   8  8.80e-01  1.00e-01  5.60e-01
   9  9.50e-01  4.00e-01  6.30e-01
  10  9.00e-01  3.30e-01  6.90e-01


In [66]:
# Os menores valores

MIN = [minimum([A[i];B[i];C[i]]) for i = 1:NP];
MIN

10-element Array{Any,1}:
 0.06
 0.24
 0.35
 0.71
 0.6 
 0.29
 0.32
 0.1 
 0.4 
 0.33

In [44]:
# As razões
RA = A./MIN; RB = B./MIN; RC = C./MIN
@printf("%4s %9s %9s %9s\n", "Prob", "r_A", "r_B", "r_C")
for i = 1:NP
    @printf("%4d %9.2f %9.2f %9.2f\n", i, RA[i], RB[i], RC[i])
end

Prob       r_A       r_B       r_C
   1      6.07      4.64      1.00
   2      1.00      1.25      1.43
   3      1.00      6.86      5.29
   4      1.16      1.84      1.00
   5      1.00      2.38      2.29
   6      1.00      7.33     15.17
   7      2.80      2.00      1.00
   8      3.73      1.00      4.13
   9      1.13      1.00      5.56
  10      5.50      8.17      1.00
  11      5.25      5.87      1.00
  12      4.05      1.00      1.24
  13      1.49      1.44      1.00
  14      4.30      1.00      1.00
  15      2.91      1.96      1.00
  16      8.38      5.38      1.00
  17      2.27      2.13      1.00
  18      2.10      1.00      1.15
  19      1.41      1.16      1.00
  20      5.87      1.00      3.44
  21      1.00      1.56      3.00
  22      4.77      1.00      6.00
  23      6.10      1.00      2.50
  24      1.00      1.25      1.37
  25      2.57     14.29      1.00
  26      1.00     10.57      6.43
  27     18.20      1.00     15.20
  28      1.16      

In [45]:
# As funções são mais complicadas
#T = zeros(NP, 3);
#T[:,1] = RA; T[:,2] = RB; T[:,3] = RC; # Cada linha é um problema
ρA(τ) = sum(RA .<= τ)/NP # Gambiarras. Se for ≦ dá 1
ρB(τ) = sum(RB .<= τ)/NP # Soma os 1
ρC(τ) = sum(RC .<= τ)/NP # Obtém o resultado

ρC (generic function with 1 method)

In [81]:
# Dados A, B e C
A[3] = Inf
M = min(A, B, C)
ra, rb, rc = A./M, B./M, C./M
t = unique(sort([ra;rb;rc]))
rhoA = [sum(ra .<= τ) for τ in t]/NP
rhoB = [sum(rb .<= τ) for τ in t]/NP
rhoC = [sum(rc .<= τ) for τ in t]/NP
plot(t, rhoA, t=:steppre)
plot!(t, rhoB, t=:steppre)
plot!(t, rhoC, t=:steppre)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 y2 
 
 
 
 y3

In [46]:
using Plots
gr()

Plots.GRBackend()

In [47]:
plot(ρA, 0, maximum(T))
plot!(ρB, 0, maximum(T))
plot!(ρC, 0, maximum(T))
xlims!(1, maximum(T))
xaxis!(:log)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0.00 
 
 
 10 
 
 
 0.25 
 
 
 10 
 
 
 0.50 
 
 
 10 
 
 
 0.75 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 -2.61332e+40,377.078 -621.258,377.078 -437.632,377.078 -330.218,377.078 -254.006,377.078 -194.892,377.078 -146.592,377.078 -105.755,377.078 -70.3804,377.078 -39.1778,377.078 
 -11.2661,377.078 13.9831,377.078 37.0338,237.099 58.2384,228.147 77.8708,211.464 96.1481,203.325 113.245,196.001 129.306,187.862 144.448,179.317 158.771,172.4 
 172.36,163.855 185.285,156.53 197.609,147.171 209.385,140.66 220.66,130.894 231.474,128.453 241.864,123.57 251.862,117.466 261.497,111.769 270.793,107.293 
 279.774,104.038 288.46,100.376 296.871,94.679 305.023,92.6444 312.932,90.2029 320.611,85.32 328.074,82.4716 335.332,79.2162 342.397,77.1817 349.278,75.554 
 355.985,72.7056 362.527,71.0779 368.911,69.8572 375.144,69.4503 381.235,68.2295 387.188,65.7881 393.011,62.9397 398.708,60.9051 404.285,59.6843 409.748,57.6498 
 415.1,57.2428 420.346,54.8014 425.49,51.953 430.536,49.5115 435.488,47.8838 440.349,45.8492 445.122,43.8147 449.811,42.187 454.419,41.7801 458.947,40.9663 
 463.4,38.5248 467.778,37.304 472.086,37.304 476.325,36.0833 480.497,34.4556 484.604,34.4556 488.649,33.6418 492.633,33.2349 496.557,32.828 500.425,32.421 
 504.237,32.0141 507.994,30.7934 511.7,29.9796 515.354,28.7588 518.958,27.945 522.514,27.1312 526.023,27.1312 529.486,25.0966 532.904,25.0966 536.279,23.8758 
 539.611,23.8758 542.902,22.6551 546.153,21.4343 549.364,21.0274 552.536,20.2136 555.672,19.8067 558.77,18.9929 561.833,18.179 564.86,17.7721 567.854,16.5514 
 570.814,15.7375 573.741,14.5168 576.636,14.5168 579.5,14.5168 582.334,13.703 585.137,13.703 587.911,13.703 590.656,12.4822 593.373,12.0753 596.063,11.2615 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 -2.61332e+40,377.078 -621.258,377.078 -437.632,377.078 -330.218,377.078 -254.006,377.078 -194.892,377.078 -146.592,377.078 -105.755,377.078 -70.3804,377.078 -39.1778,377.078 
 -11.2661,377.078 13.9831,377.078 37.0338,233.03 58.2384,220.416 77.8708,211.057 96.1481,196.815 113.245,184.2 129.306,176.062 144.448,164.668 158.771,154.495 
 172.36,149.613 185.285,147.985 197.609,142.288 209.385,134.557 220.66,128.046 231.474,121.942 241.864,115.839 251.862,112.583 261.497,108.921 270.793,104.038 
 279.774,100.376 288.46,96.7136 296.871,89.796 305.023,87.3545 312.932,83.6923 320.611,81.6577 328.074,78.8093 335.332,76.3678 342.397,71.8918 349.278,69.4503 
 355.985,66.6019 362.527,63.3466 368.911,62.1258 375.144,59.6843 381.235,57.6498 387.188,56.0221 393.011,55.2083 398.708,53.5806 404.285,51.546 409.748,50.7322 
 415.1,47.8838 420.346,44.6285 425.49,42.5939 430.536,41.7801 435.488,39.7455 440.349,38.1178 445.122,36.4902 449.811,34.4556 454.419,33.2349 458.947,32.0141 
 463.4,30.3865 467.778,29.1657 472.086,27.5381 476.325,26.7242 480.497,26.7242 484.604,25.5035 488.649,24.2827 492.633,22.6551 496.557,22.2482 500.425,21.8413 
 504.237,21.4343 507.994,20.6205 511.7,19.8067 515.354,19.8067 518.958,18.9929 522.514,18.5859 526.023,16.5514 529.486,15.7375 532.904,14.5168 536.279,14.1099 
 539.611,13.703 542.902,13.703 546.153,12.4822 549.364,12.4822 552.536,12.0753 555.672,11.2615 558.77,10.8546 561.833,10.4476 564.86,9.2269 567.854,8.81999 
 570.814,8.81999 573.741,7.59924 576.636,6.78541 579.5,6.3785 582.334,5.97158 585.137,5.97158 587.911,5.15775 590.656,4.75084 593.373,3.93701 596.063,3.93701 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#3cb370; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 -2.61332e+40,377.078 -621.258,377.078 -437.632,377.078 -330.218,377.078 -254.006,377.078 -194.892,377.078 -146.592,377.078 -105.755,377.078 -70.3804,377.078 -39.1778,377.078 


In [48]:
t = sort(T[:])
plot(t, [ρA(x) for x in t])
plot!(t, [ρB(x) for x in t])
plot!(t, [ρC(x) for x in t])
xlims!(1, maximum(T))
xaxis!(:log)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0.00 
 
 
 10 
 
 
 0.25 
 
 
 10 
 
 
 0.50 
 
 
 10 
 
 
 0.75 
 
 
 0.50 
 
 
 0.75 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 y2 
 
 
 
 y3

In [49]:
t = linspace(1, maximum(T), 100)
plot(t, [ρA(x) for x in t])
plot!(t, [ρB(x) for x in t])
plot!(t, [ρC(x) for x in t])
xlims!(1, maximum(T))
xaxis!(:log)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0.00 
 
 
 10 
 
 
 0.25 
 
 
 10 
 
 
 0.50 
 
 
 10 
 
 
 0.75 
 
 
 0.50 
 
 
 0.75 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 35.4331,357.341 54.2806,347.473 71.8759,326.503 88.3749,309.234 103.906,301.833 118.577,290.114 132.478,280.863 145.686,269.145 158.267,259.276 170.277,250.025 
 181.766,237.073 192.777,227.205 203.349,216.72 213.516,204.385 223.306,194.516 232.748,190.816 241.864,185.265 250.677,176.014 259.207,168.612 267.47,161.828 
 275.484,157.511 283.262,153.193 290.818,147.026 298.165,140.858 305.313,138.391 312.274,135.307 319.056,129.14 325.669,124.206 332.121,119.271 338.42,116.804 
 344.572,114.954 350.585,111.87 356.464,108.17 362.215,105.703 367.845,103.852 373.357,103.236 378.757,102.619 384.049,100.152 389.237,96.4513 394.326,92.1339 
 399.318,90.2837 404.219,88.4334 409.03,85.3496 413.756,84.7328 418.399,81.649 422.961,77.9484 427.447,75.4814 431.858,71.7808 436.196,70.5473 440.465,67.4635 
 444.666,64.9964 448.801,62.5294 452.873,61.2959 456.883,61.2959 460.834,60.0623 464.726,56.3618 468.562,54.5115 472.343,54.5115 476.071,52.6612 479.748,50.1941 
 483.374,50.1941 486.951,49.5774 490.48,48.9606 493.963,48.3439 497.401,47.7271 500.795,47.1103 504.145,46.4936 507.454,44.6433 510.722,43.4098 513.951,42.793 
 517.14,41.5595 520.292,39.7092 523.406,39.0924 526.484,38.4757 529.527,36.0086 532.536,36.0086 535.51,34.7751 538.452,34.1583 541.361,34.1583 544.238,31.6913 
 547.085,30.4578 549.902,29.841 552.688,28.6075 555.446,28.6075 558.176,26.7572 560.877,25.5237 563.551,25.5237 566.199,23.6734 568.82,23.0566 571.416,20.5896 
 573.986,19.9728 576.532,19.9728 579.054,19.9728 581.552,18.7393 584.026,18.7393 586.478,18.7393 588.907,18.7393 591.314,16.889 593.699,16.2722 596.063,15.0387 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 35.4331,351.174 54.2806,333.288 71.8759,324.036 88.3749,306.767 103.906,285.797 118.577,272.845 132.478,258.66 145.686,246.324 158.267,233.372 170.277,225.354 
 181.766,224.121 192.777,217.953 203.349,208.702 213.516,200.067 223.306,188.966 232.748,181.564 241.864,173.546 250.677,169.229 259.207,164.912 267.47,157.511 
 275.484,152.577 283.262,146.409 290.818,142.708 298.165,134.074 305.313,130.373 312.274,125.439 319.056,121.739 325.669,119.888 332.121,114.337 338.42,110.637 
 344.572,105.086 350.585,101.385 356.464,98.3016 362.215,94.601 367.845,92.1339 373.357,89.6669 378.757,87.1998 384.049,84.7328 389.237,82.8825 394.326,80.4155 
 399.318,79.1819 404.219,76.0981 409.03,74.8646 413.756,71.7808 418.399,68.697 422.961,63.7629 427.447,61.9126 431.858,60.0623 436.196,58.212 440.465,55.745 
 444.666,53.2779 448.801,51.4277 452.873,48.9606 456.883,46.4936 460.834,45.8768 464.726,44.0265 468.562,41.5595 472.343,39.0924 476.071,38.4757 479.748,38.4757 
 483.374,36.6254 486.951,36.0086 490.48,32.9248 493.963,31.6913 497.401,31.6913 500.795,31.0745 504.145,30.4578 507.454,29.2242 510.722,27.9907 513.951,27.9907 
 517.14,27.9907 520.292,26.1404 523.406,25.5237 526.484,23.0566 529.527,21.8231 532.536,20.5896 535.51,19.9728 538.452,18.7393 541.361,18.7393 544.238,18.1225 
 547.085,16.889 549.902,16.889 552.688,16.2722 555.446,15.0387 558.176,14.422 560.877,13.8052 563.551,11.9549 566.199,11.9549 568.82,11.3381 571.416,11.3381 
 573.986,9.48786 576.532,8.25434 579.054,7.63758 581.552,7.02082 584.026,7.02082 586.478,5.78729 588.907,5.78729 591.314,5.17053 593.699,3.93701 596.063,3.93701 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#3cb370; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 35.4331,377.078 54.2806,365.359 71.8759,348.09 88.3749,336.988 103.906,322.803 118.577,311.084 132.478,293.815 145.686,282.713 158.267,266.678 170.277,258.66 
 181.766,252.492 192.777,245.708 203.349,228.438 213.

In [50]:
t = linspace(1, maximum(T), 1000)
plot(t, [ρA(x) for x in t])
plot!(t, [ρB(x) for x in t])
plot!(t, [ρC(x) for x in t])
xlims!(1, maximum(T))
xaxis!(:log)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0.00 
 
 
 10 
 
 
 0.25 
 
 
 10 
 
 
 0.50 
 
 
 10 
 
 
 0.75 
 
 
 0.50 
 
 
 0.75 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 35.4331,357.341 37.3619,357.341 39.2767,356.725 41.1778,355.491 43.0654,354.258 44.9396,352.407 46.8006,351.174 48.6486,349.94 50.4839,348.707 52.3065,347.473 
 54.1167,347.473 55.9146,345.623 57.7003,343.773 59.4741,341.922 61.2362,341.306 62.9865,340.072 64.7254,337.605 66.4529,336.372 68.1693,332.054 69.8746,330.821 
 71.569,326.503 73.2526,324.653 74.9256,320.953 76.5881,319.102 78.2402,318.485 79.8821,317.252 81.5139,317.252 83.1356,314.785 84.7475,312.935 86.3497,311.701 
 87.9422,309.851 89.5253,308.617 91.0989,308.001 92.6632,308.001 94.2183,307.384 95.7644,306.767 97.3015,305.534 98.8297,304.917 100.349,304.3 101.86,303.683 
 103.362,302.45 104.856,301.216 106.341,300.599 107.818,299.983 109.287,298.132 110.748,297.516 112.201,295.049 113.646,295.049 115.083,294.432 116.512,293.198 
 117.934,291.348 119.348,289.498 120.754,289.498 122.153,287.031 123.545,286.414 124.93,285.18 126.307,284.564 127.677,283.33 129.04,282.713 130.396,281.48 
 131.745,280.863 133.088,279.63 134.423,278.396 135.752,276.546 137.075,274.695 138.39,274.695 139.699,273.462 141.002,273.462 142.299,272.228 143.589,269.761 
 144.873,269.761 146.15,269.145 147.422,268.528 148.687,267.911 149.947,267.294 151.2,266.678 152.448,264.21 153.689,261.743 154.925,261.743 156.155,261.743 
 157.38,260.51 158.599,259.276 159.812,258.66 161.02,256.809 162.222,255.576 163.419,254.959 164.61,254.959 165.796,253.726 166.977,252.492 168.153,251.258 
 169.323,250.025 170.488,250.025 171.648,246.941 172.804,245.708 173.954,244.474 175.099,243.241 176.239,243.241 177.374,240.774 178.505,240.774 179.63,240.157 
 180.751,240.157 181.867,237.073 182.979,236.456 184.085,235.839 185.188,235.223 186.285,233.989 187.378,233.372 188.467,230.905 189.551,230.289 190.631,230.289 
 191.706,229.672 192.777,227.205 193.844,223.504 194.907,222.887 195.965,222.271 197.019,221.037 198.069,219.804 199.114,219.187 200.156,218.57 201.193,217.337 
 202.227,216.72 203.256,216.72 204.281,216.103 205.303,214.87 206.32,213.636 207.334,213.019 208.344,211.786 209.35,211.169 210.352,209.935 211.35,206.235 
 212.345,205.618 213.336,204.385 214.323,203.768 215.307,203.768 216.287,202.534 217.263,202.534 218.236,202.534 219.205,196.367 220.17,196.367 221.132,196.367 
 222.091,196.367 223.046,194.516 223.998,194.516 224.946,194.516 225.891,194.516 226.833,193.9 227.771,193.283 228.706,192.666 229.638,192.666 230.566,192.666 
 231.491,192.666 232.413,190.816 233.332,190.816 234.247,189.582 235.16,189.582 236.069,188.966 236.975,188.349 237.878,187.732 238.778,187.732 239.675,187.732 
 240.569,186.498 241.46,185.882 242.348,185.265 243.233,184.648 244.115,184.031 244.994,181.564 245.87,181.564 246.744,180.331 247.614,180.331 248.482,179.097 
 249.346,177.247 250.208,177.247 251.068,176.014 251.924,176.014 252.778,175.397 253.628,173.546 254.477,172.93 255.322,171.696 256.165,171.079 257.005,171.079 
 257.842,169.229 258.677,168.612 259.509,168.612 260.339,167.379 261.166,167.379 261.99,166.762 262.812,166.145 263.631,166.145 264.448,164.912 265.262,164.912 
 266.074,164.295 266.883,163.062 267.69,161.828 268.494,161.828 269.296,161.211 270.096,161.211 270.893,160.594 271.688,160.594 272.48,160.594 273.27,160.594 
 274.058,159.978 274.843,158.744 275.626,156.894 276.406,156.277 277.185,156.277 277.961,156.277 278.735,155.66 279.506,155.66 280.276,155.044 281.043,154.427 
 281.808,154.427 282.57,153.193 283.331,153.193 284.089,153.193 284.845,153.193 285.599,153.193 286.351,151.96 287.101,150.11 287.848,150.11 288.594,148.876 
 289.337,148.876 290.079,147.026 290.818,147.026 291.555,146.409 292.29,145.792 293.023,143.942 293.755,143.325 294.484,142.708 295.211,142.708 295.

In [62]:
t = sort([RA; RB; RC])
plot(t, [ρA(x) for x in t], t=:steppre)
plot!(t, [ρB(x) for x in t], t=:steppre)
plot!(t, [ρC(x) for x in t], t=:steppre)
xlims!(0.99, maximum([RA; RB; RC]))
ylims!(0.0, 1.01)
xaxis!(:log)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0.0 
 
 
 10 
 
 
 0.5 
 
 
 10 
 
 
 1.0 
 
 
 10 
 
 
 1.5 
 
 
 10 
 
 
 2.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 36.6539,249.988 36.6539,249.988 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 36.6539,249.619 
 38.1819,249.619 38.2845,249.619 38.4018,249.619 38.7305,249.619 38.7305,249.249 38.804,249.249 
 39.1586,249.249 39.1586,248.88 39.2114,248.88 39.2114,248.511 39.5123,248.511 39.5123,248.141 39.5812,248.141 
 39.6534,248.141 40.076,248.141 40.076,247.772 40.227,247.772 
 40.5106,247.772 40.5106,247.402 40.6811,247.402 40.7034,247.402 40.7034,247.033 40.7721,247.033 
 40.7721,246.663 41.0717,246.663 41.2384,246.663 41.3567,246.663 
 41.4813,246.663 41.4813,246.294 41.5462,246.294 41.5462,245.555 42.0537,245.555 
 42.1764,245.555 42.2186,245.555 42.3049,245.555 42.4395,245.555 
 42.5807,245.555 42.7289,245.555 
 42.8847,245.555 43.2217,245.555 43.2217,245.186 43.3423,245.186 43.4042,245.186 43.4042,244.816 43.5187,244.816 
 43.5972,244.816 43.6979,244.816 44.0183,244.816 44.1087,244.816 44.2485,244.816 44.2485,244.447 
 44.2485,244.447 44.3285,244.447 44.3285,244.077 44.3692,244.077 44.4937,244.077 
 44.6661,244.077 
 44.8463,244.077 44.9395,244.077 45.2321,244.077 45.2321,243.338 45.3343,243.338 
 45.4923,243.338 45.6561,243.338 45.6561,242.969 45.8261,242.969 45.8261,242.599 
 46.0544,242.599 46.0544,242.23 46.2238,242.23 46.5757,242.23 46.6437,242.23 46.6437,241.86 
 46.6781,241.86 46.9984,241.86 47.0427,241.86 47.0427,241.491 47.2236,241.491 47.2236,240.752 
 47.3567,240.752 47.4109,240.752 47.7046,240.752 
 48.0149,240.752 48.076,240.752 48.076,240.383 48.2316,240.383 48.2316,240.383 48.5145,240.383 
 48.6118,240.383 48.6118,240.013 48.6611,240.013 48.7442,240.013 48.7442,239.644 48.8115,239.644 49.0782,239.644 
 49.1881,239.644 49.1881,239.274 49.4525,239.274 49.4525,238.905 49.6815,238.905 49.6815,238.535 
 49.7596,238.535 49.7991,238.535 49.7991,238.166 49.8501,238.166 49.9188,238.166 50.0819,238.166 
 50.165,238.166 50.165,237.797 50.4204,237.797 50.4204,237.427 50.5518,237.427 50.5518,235.949 
 50.9615,235.949 
 50.9615,235.21 50.9615,235.21 50.9615,234.841 51.2484,234.841 
 51.275,234.841 51.3071,234.841 51.5469,234.841 51.9451,234.841 51.9451,234.472 51.9953,234.472 
 52.1824,234.472 52.2776,234.472 52.3498,234.472 52.3498,234.102 52.4064,234.102 52.4064,233.733 52.6958,233.733 
 52.6958,232.994 52.8745,232.994 52.8745,232.255 52.8745,232.255 
 52.8745,231.885 53.0742,231.885 53.0742,231.516 53.1905,231.516 53.2443,231.516 
 53.2788,231.516 53.2788,230.408 53.4356,230.408 53.6314,230.408 
 53.8608,230.408 53.8608,229.669 54.037,229.669 54.037,229.299 54.2168,229.299 
 54.3541,229.299 54.3541,228.191 54.4625,228.191 
 54.6831,228.191 54.6831,227.822 54.8182,227.822 54.8678,227.822 54.975,227.822 55.1413,227.822 
 55.1749,227.822 55.3792,227.822 55.3792,227.083 55.3792,227.083 
 55.7062,227.083 55.7479,227.083 55.8018,227.083 55.8018,226.713 55.9762,226.713 55.9762,225.974 
 56.3958,225.974 56.5307,225.974 56.5621,225.974 56.6125,225.974 
 56.7596,225.974 56.9467,225.974 57.2923,225.974 
 57.5293,225.974 57.7495,225.974 57.8334,225.974 57.9367,225.974 58.0201,225.974 
 58.1464,225.974 58.1464,225.605 58.1954,225.605 58.3604,225.605 58.3604,225.235 58.4037,225.235 58.4037,224.866 
 58.4689,224.866 58.4689,224.497 58.4941,224.497 58.4941,223.388 58.8013,223.388 
 59.

## Analisando

Os resultados são aleatório, então vamos analisar de maneira geral.
O algoritmo mais acima é melhor. O mais acima à esquerda é o mais
eficiente, isto é, aquele que resolve os problemas mais rápido que os outros.
O mais acima à direta é o mais robusto, isto é, aquele que resolve
mais problemas. Note que neste caso todos convergem par todos os problemas.

Na abscissa, temos a razão com qual o algoritmo converge, e o valor
da função nesse ponto quer dizer "dado x vezes mais tempo que o melhor
algoritmo, quantos problemas o algoritmo resolve".
Parece um pouco complicado de entender, mas a ideia é a seguinte.
O algoritmo A tem os tempos [1s 2s 10s], e o B tem os tempos
[2s 3s 12s].
Então, o algoritmo A é o melhor e resolve 3 problemas no instante inicial.
Para $\tau = 1$, o algoritmo B resolveu 0 problemas. Matematicamente,
as razões de B são [2 1.5 1.2], então a função acumulada é 0.

Para $\tau = 1.2$, queremos saber quantos problemas o algoritmo em
até 20% do melhor algoritmo. Os melhores tempos são [1s 2s 10s],
e 20% a mais disso seria [1.2s 2.4s 12s]. Vemos que B consegue
resolver o terceiro problemas em 12s, então a função acumulada pega esse valor.
Em até 50% a mais de tempo, o algoritmo B resolve 2 problemas, e em até
o dobro do tempo, o algoritmo B resolve 3 problemas.


## Gerando o performance profile

Como vimos no exemplo, não é muito fácil gerar o performance profile, e portanto,
desde que foi inventado, várias pessoas fizeram códigos que geram esse gráfico
para o usuário.

Eu recomendo o [perprof-py](https://github.com/ufpr-opt/perprof-py), que
eu ajudei a implementar. Para esse software, devemos providenciar os resultados
dos algoritmos em arquivos de texto em formato de tabela.

Um novo pacote, nativo do Julia para isso é o [Profiles.jl](https://github.com/JuliaSmoothOptimizers/Profiles.jl).

In [53]:
Pkg.clone("https://github.com/JuliaSmoothOptimizers/Profiles.jl")
Pkg.build("Profiles")
using Profiles

INFO: Cloning Profiles from https://github.com/JuliaSmoothOptimizers/Profiles.jl
INFO: Computing changes...
INFO: No packages to install, update or remove
INFO: Building Plots


In [83]:
A = [Inf; Inf; Inf; Inf; Inf; ones(5)]
performance_profile([A B C], ["A", "B", "C"])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 Within this factor of the best (log scale) 
 
 
 Proportion of problems 
 
 
 
 
 
 
 
 
 A 
 
 
 
 B 
 
 
 
 C

# Pacotes de Testes

- [NLPModels.jl](https://github.com/JuliaSmoothOptimizers/NLPModels.jl)
- [CUTEst.jl](https://github.com/JuliaSmoothOptimizers/CUTEst.jl)
- [OptimizationProblems.jl](https://github.com/JuliaSmoothOptimizers/OptimizationProblems.jl)

In [86]:
Pkg.rm("CUTEst")
Pkg.clone("https://github.com/JuliaSmoothOptimizers/CUTEst.jl")
Pkg.build("CUTEst")

INFO: Removing CUTEst (unregistered)
INFO: Cloning CUTEst from https://github.com/JuliaSmoothOptimizers/CUTEst.jl
INFO: Computing changes...


In [87]:
using CUTEst